In [1]:
from os import listdir
import os
import json
import pandas as pd
from matplotlib.pyplot import xlabel
from typing import List, Dict

import pandas as pd

In [2]:
def load_solutions(output_dir: str) -> List[Dict]:
    file_names = os.listdir(output_dir)
    upv_files = [f for f in file_names if f.startswith("UPV_")]
    urjc_files = [f for f in file_names if f not in upv_files]

    urjc_solutions = load_json_solutions([os.path.join(output_dir, f) for f in urjc_files])
    upv_solutions = load_json_solutions([os.path.join(output_dir, f) for f in upv_files])

    return upv_solutions, urjc_solutions

def load_json_solutions(file_paths: List[str]) -> List[Dict]:
    return [json.load(open(path)) for path in file_paths]

def get_best_solution(solutions: List[Dict]) -> Dict:
    return min(solutions, key=lambda x: x['computational_time'])
    #return min(solutions, key=lambda x: x['cost'])

def extract_number_by_string(input_string: List[str], pattern : str) -> List[int]:
    rqt_numbers = []
    groups_letters = input_string.split("_")
    if not pattern:
        return groups_letters[-1]
    
    for group in groups_letters:
        if pattern in group:
            end = group.find(pattern)
            return int(group[:end])

    return "Not Found"

def extract_results(results_json, exp_graph, f, algorithm):
    
        used_vehicles_costs = [cost for cost in results_json["costs_routes"].values() if cost > 0]
        
        return {
            "graph" : exp_graph,
            "instance" : f,
            "algorithm": algorithm,
            "requests": extract_number_by_string(results_json["instance_name"], "RQT"),
            "robots" : extract_number_by_string(results_json["instance_name"], "HR"),
            "capacity" : extract_number_by_string(results_json["instance_name"], "Q"),
            "demand" : extract_number_by_string(results_json["instance_name"], "q"),
            "num_instance" : extract_number_by_string(results_json["instance_name"], None),
            "isBigTW": bool("bigTW" in results_json["instance_name"]),
            "cost": results_json['cost'],
            "time": results_json['max_time'],
            "distance" : results_json['distance_travelled'],
            "used_vehicles": len(used_vehicles_costs),
            "used_max": max(used_vehicles_costs),
            "used_min": min(used_vehicles_costs),
            "used_mean": sum(used_vehicles_costs) / len(used_vehicles_costs),
            "computational_time" : results_json['computational_time']
        }

def add_empty_results(exp_graph, f, algorithm):

        return {
            "graph" : exp_graph,
            "instance" : f,
            "algorithm": algorithm,
            "requests": extract_number_by_string(f, "RQT"),
            "robots" : extract_number_by_string(f, "HR"),
            "capacity" : extract_number_by_string(f, "Q"),
            "demand" : extract_number_by_string(f, "q"),
            "num_instance" : extract_number_by_string(f, None),
            "isBigTW": bool("bigTW" in f),
            "cost": None,
            "time": None,
            "distance" : None,
            "used_vehicles": None,
            "used_max": None,
            "used_min": None,
            "used_mean": None,
            "computational_time" : None
        }

In [3]:
experiments_graph =  ["mountain", "random", "big_vid", "big_vid_2"]

all_results = []
fail_experiments = []
for exp_graph in experiments_graph:
    path = f"{exp_graph}" # Path to the folder containing the experiments
    exp_files = listdir(path)
    for experiment in exp_files:
        upv_solutions, urjc_solutions = load_solutions(f"{path}/{experiment}/output")

        try:
            best_urjc = get_best_solution(urjc_solutions)
            urjc_results = extract_results(best_urjc, exp_graph,  experiment, "MILP")
            all_results.append(urjc_results)
        except ValueError:
            urjc_results = add_empty_results(exp_graph, experiment, "MILP")
            all_results.append(urjc_results)
            print(f"Error: No MILP solutions found for {path}/{experiment}")
            fail_experiments.append(f"{path}/{experiment}")
            continue

        best_upv = get_best_solution(upv_solutions)
        upv_results = extract_results(best_upv, exp_graph, experiment, "Heuristic")
        
        all_results.append(upv_results)
results_df = pd.DataFrame(all_results)
results_df.to_csv("results.csv", index=False)

Error: No MILP solutions found for big_vid/9HR_6Q_100RQT_2q_bigTW_2
Error: No MILP solutions found for big_vid/9HR_6Q_100RQT_3q_bigTW_2
Error: No MILP solutions found for big_vid/9HR_6Q_100RQT_3q_bigTW_5
Error: No MILP solutions found for big_vid/9HR_6Q_100RQT_2q_bigTW_5
Error: No MILP solutions found for big_vid/6HR_6Q_50RQT_2q_bigTW_2
Error: No MILP solutions found for big_vid/6HR_6Q_50RQT_3q_bigTW_2
Error: No MILP solutions found for big_vid/3HR_6Q_100RQT_3q_bigTW_5
Error: No MILP solutions found for big_vid/6HR_6Q_50RQT_3q_bigTW_5
Error: No MILP solutions found for big_vid/6HR_6Q_50RQT_2q_bigTW_5
Error: No MILP solutions found for big_vid/3HR_6Q_100RQT_3q_bigTW_2
Error: No MILP solutions found for big_vid/6HR_6Q_50RQT_3q_bigTW_4
Error: No MILP solutions found for big_vid/6HR_6Q_50RQT_2q_bigTW_4
Error: No MILP solutions found for big_vid/3HR_6Q_100RQT_3q_bigTW_3
Error: No MILP solutions found for big_vid/6HR_6Q_50RQT_2q_bigTW_3
Error: No MILP solutions found for big_vid/6HR_6Q_50RQT